### Script for creating images for all dates - run only one time

In [1]:
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
# load geometry data
vic_geo_df = gpd.read_file('../data/VIC_LOCALITY_POLYGON_shp.geojson')

In [3]:
# load active cases data
df_all_dates = pd.read_csv('../data/all_dates.csv')

In [4]:
# merge geometry with active cases
vic_geo_df = pd.merge(vic_geo_df[['VIC_LGA__3', 'geometry']], df_all_dates, how = 'inner', 
                      left_on = 'VIC_LGA__3', right_on = 'lga', sort = True)

In [5]:
# create colour map that will be used for all images
cmap = col.LinearSegmentedColormap.from_list(
    'covid_custom', [(0,'white'), (0.01, '#fffa86'), (0.25, 'yellow'), (0.5, 'red'), (0.75, 'brown'), (1, 'black')])

In [6]:
# create list of dates
date_list = vic_geo_df.columns.tolist()

date_list.remove('VIC_LGA__3')
date_list.remove('geometry')
date_list.remove('lga')

In [7]:
# set vmax for visualisation scale
max_cases = vic_geo_df[date_list].max().max()
vmax = math.ceil(max_cases/100) * 100

# set number of bins such that bin size is 10
k = vmax/10

In [8]:
# define paths where image files will be stored
MELB_CASES_PATH = '../data/images/melb/cases_absolute/'
VIC_CASES_PATH = '../data/images/all_vic/cases_absolute/'

In [9]:
# define function for visualising one date
def create_image(gdf, col_name, path, img_num):
    covid_map = gdf.plot(figsize = (14,10), column = col_name, cmap = cmap, k = k, 
                         vmin = 0, vmax = vmax, edgecolor = 'lightblue')
    covid_map.axis('off')
    plt.tight_layout()
    plt.savefig(path + str(img_num).zfill(3) + '.png')
    plt.close()

#### first map just the melb lgas

In [10]:
melb_lgas = ['Banyule','Bayside','Boroondara','Brimbank','Cardinia','Casey','Darebin','Frankston','Glen Eira',
'Greater Dandenong','Hobsons Bay','Hume','Kingston','Knox','Manningham','Maribyrnong','Maroondah','Melbourne',
'Melton','Monash','Moonee Valley','Moreland','Mornington Peninsula','Nillumbik','Port Phillip','Stonnington',
'Whitehorse','Whittlesea','Wyndham','Yarra','Yarra Ranges']

melb_lgas = [lga.upper() for lga in melb_lgas]

In [11]:
melb_geo_df = vic_geo_df[vic_geo_df['lga'].isin(melb_lgas)]

In [12]:
for i in range(len(date_list)):
    create_image(melb_geo_df, date_list[i], MELB_CASES_PATH, i)

#### now map all victoria

In [13]:
for i in range(len(date_list)):
    create_image(vic_geo_df, date_list[i], VIC_CASES_PATH, i)

#### save color scale in its own image file

In [14]:
ax = plt.subplot()
plt.colorbar(cm.ScalarMappable(norm=col.Normalize(0, vmax), cmap=cmap), ax=ax)
ax.remove()
plt.savefig('../data/images/scale.svg', bbox_inches = 'tight')
plt.close()